In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan
import numpy as np
import pandas as pd
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

In [ ]:
# import MetaTrader5 as mt5
# # from vectorbt.portfolio import Portfolio as vbt_Portfolio

# Connection

In [ ]:
# RICO_REAL   = { 'login':16331371,   'password':'#582640Zz', 'server':'Rico-PRD'                 }
# CLEAR_REAL  = { 'login':1018156111, 'password':'#582640Zz', 'server':'ClearInvestimentos-CLEAR' }

# PATH_7 = r'C:\Program Files\MetaTrader 5 _7\terminal64.exe'

# assert mt5.initialize(PATH_7, **CLEAR_REAL)

# Variables

In [ ]:
SYMBOLS = ['WIN$'] # ['WDO$']
TFRAMES = ['M5','M15','H1','H4','D1']

# Load

In [ ]:
# def MT5_LOAD_DATA(SYMBOLS, TFRAMES, start_pos=0, count=99_999):
#     Pipe = []
#     for (symbol, TF) in it.product(SYMBOLS, TFRAMES):
#         timeframe = { 'M1':mt5.TIMEFRAME_M1, 'M5':mt5.TIMEFRAME_M5, 'M10':mt5.TIMEFRAME_M10, 'M15':mt5.TIMEFRAME_M15, 'M20':mt5.TIMEFRAME_M20, 'M30':mt5.TIMEFRAME_M30, 'H1':mt5.TIMEFRAME_H1, 'H2':mt5.TIMEFRAME_H2, 'H3':mt5.TIMEFRAME_H3, 'H4':mt5.TIMEFRAME_H4, 'H6':mt5.TIMEFRAME_H6, 'H8':mt5.TIMEFRAME_H8, 'H12':mt5.TIMEFRAME_H12, 'D1':mt5.TIMEFRAME_D1, 'W1':mt5.TIMEFRAME_W1 }[TF]
#         Raw       = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, start_pos, count))
#         Raw.insert(0, 'a',          Raw.index + 1)
#         Raw.insert(1, 'z',          Raw.index - Raw.index.max())
#         Raw.insert(2, 'symbol',     symbol)
#         Raw.insert(3, 'tf',         TF)
#         Raw.insert(4, 'datetime',   pd.to_datetime(Raw['time'], unit='s'))
#         Pipe.append(Raw)
#     pass
#     return pd.concat(Pipe, ignore_index=1)

# Raw = MT5_LOAD_DATA(SYMBOLS, TFRAMES)
# Raw.head()

In [ ]:
# Raw.to_csv('Storage/Raw.csv', index=False)
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['D1'])
Query.head()

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['A']        = Query['a']
    Src['Z']        = Query['z']
    Src['Symbol']   = Query['symbol']
    Src['TF']       = Query['tf']
    
    Src['Datetime'] = Query['datetime']
    Src['Date']     = Query['datetime'].dt.date
    Src['Time']     = Query['datetime'].dt.time
    
    Src['Ticks']    = Query['tick_volume']
    Src['Volume']   = Query['real_volume']
    Src['Price']    = Query['close']
    
    Src['Open']     = Query['open']
    Src['High']     = Query['high']
    Src['Low']      = Query['low']
    Src['Close']    = Query['close']

    Src['Change']   = Src['Close'] - Src['Open']
    Src['HL']       = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

# Data

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)

    for x in [100, 200, 300, 500, 800, 1000, 1200, 1500, 1800, 2000, 2200, 2500, 2800, 3000, 3200, 3500, 3800, 4000, 4200, 4500, 4800, 5000]:
        Calc[f'Chg Abs < {x}'] = (Calc['Change'].abs() < x).astype(int)

    Calc['Chg +1']  = Calc.groupby(['Symbol','TF'], sort=0)['Change'].shift(-1)
    Calc['Chg +2']  = Calc.groupby(['Symbol','TF'], sort=0)['Change'].shift(-2)
    
    Calc['HL +1']   = Calc.groupby(['Symbol','TF'], sort=0)['Change'].shift(-1)
    Calc['HL +2']   = Calc.groupby(['Symbol','TF'], sort=0)['Change'].shift(-2)
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Markov Chain 1

In [13]:
def MARKOV_CHAIN_1(Calc, when, _next):
    pipe = []
    for x in when: 
        Df = Calc[Calc[x]==1]

        for y in _next: 

            pipe.append({ 'When': x, 'Next':y, 
                'Abs Avg':  Df[y].mean()          .round(0).astype(int),
                'Abs Max':  Df[y].max()           .round(0).astype(int),
                'Abs Q3':   Df[y].quantile(.75)   .round(0).astype(int),
                'Abs Med':  Df[y].median()        .round(0).astype(int),
                'Abs Q1':   Df[y].quantile(.25)   .round(0).astype(int),
                'Abs Min':  Df[y].min()           .round(0).astype(int),
            })
        pass
    pass
    return pd.DataFrame(pipe)

Mkv11 = MARKOV_CHAIN_1(Calc, when=Calc.loc[:, f'Chg Abs < {100}':f'Chg Abs < {5000}'].columns, _next=['Chg +1'])
Mkv11

,When,Next,Abs Avg,Abs Max,Abs Q3,Abs Med,Abs Q1,Abs Min
0,Chg Abs < 100,Chg +1,221,3536,1169,84,-814,-3306
1,Chg Abs < 200,Chg +1,86,5178,1168,-26,-1024,-4067
2,Chg Abs < 300,Chg +1,70,5397,1117,-76,-1037,-4650
3,Chg Abs < 500,Chg +1,-44,5678,980,-150,-1115,-4650
4,Chg Abs < 800,Chg +1,-99,5678,965,-129,-1160,-5935
5,Chg Abs < 1000,Chg +1,-53,5678,992,-12,-1114,-5935
6,Chg Abs < 1200,Chg +1,-31,5678,987,15,-1085,-5935
7,Chg Abs < 1500,Chg +1,-44,5678,970,6,-1100,-5935
8,Chg Abs < 1800,Chg +1,-59,5678,970,0,-1108,-5935
9,Chg Abs < 2000,Chg +1,-43,5678,989,26,-1096,-5935


In [14]:
Mkv12 = MARKOV_CHAIN_1(Calc, when=Calc.loc[:, f'Chg Abs < {100}':f'Chg Abs < {5000}'].columns, _next=['Chg +1','Chg +2'])
Mkv12

,When,Next,Abs Avg,Abs Max,Abs Q3,Abs Med,Abs Q1,Abs Min
0,Chg Abs < 100,Chg +1,221,3536,1169,84,-814,-3306
1,Chg Abs < 100,Chg +2,48,5397,780,-227,-939,-4650
2,Chg Abs < 200,Chg +1,86,5178,1168,-26,-1024,-4067
3,Chg Abs < 200,Chg +2,-108,5397,838,-132,-986,-4905
4,Chg Abs < 300,Chg +1,70,5397,1117,-76,-1037,-4650
5,Chg Abs < 300,Chg +2,-76,5397,878,-130,-1054,-4905
6,Chg Abs < 500,Chg +1,-44,5678,980,-150,-1115,-4650
7,Chg Abs < 500,Chg +2,-11,5397,950,13,-1043,-5367
8,Chg Abs < 800,Chg +1,-99,5678,965,-129,-1160,-5935
9,Chg Abs < 800,Chg +2,-49,5442,872,0,-1044,-5367


# Markov Chain 2

In [ ]:
def MARKOV_CHAIN_2(Calc, when, _next):
    pipe = []
    for x in when: 
        Df = Calc[Calc[x]==1]

        for y in _next: 

            pipe.append({ 'When': x, 'Next':y, 
                'Abs Avg':  Df[y].abs().mean()          .round(0).astype(int),
                'Abs Max':  Df[y].abs().max()           .round(0).astype(int),
                'Abs Q3':   Df[y].abs().quantile(.75)   .round(0).astype(int),
                'Abs Med':  Df[y].abs().median()        .round(0).astype(int),
                'Abs Q1':   Df[y].abs().quantile(.25)   .round(0).astype(int),
                'Abs Min':  Df[y].abs().min()           .round(0).astype(int),
            })
        pass
    pass
    return pd.DataFrame(pipe)

Mkv21 = MARKOV_CHAIN_2(Calc, when=Calc.loc[:, f'Chg Abs < {100}':f'Chg Abs < {5000}'].columns, _next=['Chg +1'])
Mkv21

,When,Next,Abs Avg,Abs Max,Abs Q3,Abs Med,Abs Q1,Abs Min
0,Chg Abs < 100,Chg +1,1207,3536,1774,1030,333,74
1,Chg Abs < 200,Chg +1,1247,5178,1691,1102,437,6
2,Chg Abs < 300,Chg +1,1288,5397,1732,1101,482,6
3,Chg Abs < 500,Chg +1,1315,5678,1772,1100,550,6
4,Chg Abs < 800,Chg +1,1331,5935,1840,1104,540,0
5,Chg Abs < 1000,Chg +1,1308,5935,1803,1074,508,0
6,Chg Abs < 1200,Chg +1,1286,5935,1800,1042,498,0
7,Chg Abs < 1500,Chg +1,1311,5935,1829,1044,506,0
8,Chg Abs < 1800,Chg +1,1301,5935,1836,1045,500,0
9,Chg Abs < 2000,Chg +1,1303,5935,1831,1047,503,0


In [ ]:
Mkv22 = MARKOV_CHAIN_2(Calc, when=Calc.loc[:, f'Chg Abs < {100}':f'Chg Abs < {5000}'].columns, _next=['Chg +1','Chg +2'])
Mkv22

,When,Next,Abs Avg,Abs Max,Abs Q3,Abs Med,Abs Q1,Abs Min
0,Chg Abs < 100,Chg +1,1207,3536,1774,1030,333,74
1,Chg Abs < 100,Chg +2,1273,5397,1590,914,480,16
2,Chg Abs < 200,Chg +1,1247,5178,1691,1102,437,6
3,Chg Abs < 200,Chg +2,1221,5397,1657,927,457,0
4,Chg Abs < 300,Chg +1,1288,5397,1732,1101,482,6
5,Chg Abs < 300,Chg +2,1304,5397,1783,993,493,0
6,Chg Abs < 500,Chg +1,1315,5678,1772,1100,550,6
7,Chg Abs < 500,Chg +2,1282,5397,1758,1006,477,0
8,Chg Abs < 800,Chg +1,1331,5935,1840,1104,540,0
9,Chg Abs < 800,Chg +2,1265,5442,1771,983,442,0
